In [ ]:
# Add your own key
# %env OPENAI_API_KEY=

In [ ]:
%env HF_DATASETS_CACHE="/data/users/sgarg6/hf_cache"

In [ ]:
models = ["text-davinci-001", "text-davinci-002", "text-davinci-003"]

# SETUP Open AI model

In [ ]:
from langchain import OpenAI
openai = OpenAI(model_name="text-davinci-003", temperature=0.7)

In [ ]:
from datasets import load_dataset

Load Dataset

In [ ]:
bold = load_dataset("AlexaAI/bold", split="train")

In [ ]:
race = bold.filter(lambda x: x["domain"] == "race")

In [ ]:
race = race.train_test_split(test_size = 0.06)

In [ ]:
race["test"]

In [ ]:
from collections import Counter

words = race["test"]["category"]

print(Counter(words).keys()) # equals to list(set(words))
Counter(words).values() # counts the elements' frequency

In [ ]:
race["test"]["prompts"]

In [ ]:
def comp(x):
    x["prompts"] = x["prompts"][0]
    x["wikipedia"] = x["wikipedia"][0]
    return x

race["test"] = race["test"].map(comp)

In [ ]:
race["test"][:2]

# Generate output for prompts from OpenAI model using LangChain

In [ ]:
def compute_continuations(x):
    x["continuations_002"] = openai.generate([x["prompts"]]).generations[0][0].text
    print(x["continuations_002"])
    return x
race["test"] = race["test"].map(compute_continuations)

# Compute Regard

In [ ]:
import evaluate 
regard = evaluate.load('regard', 'default')

In [ ]:
regard.compute(data=race["test"][f"continuations"][:1])

In [ ]:
import statistics
def compute_regard_stats(x):
    across_model_regard = []
    for mod in ["_001", "_002", ""]:
        sample_regard= regard.compute(data=[x[f"continuations{mod}"]])
        across_model_regard.append(sample_regard["regard"][0][0]["score"])
    
    if statistics.stdev(across_model_regard) > 0.2:
        print(x)
race["test"].map(compute_regard_stats)

In [ ]:
def compute_regard(x):
    for mod in ["_001", "_002", ""]:
        x[f"regard_{mod}"] = regard.compute(data=[x[f"continuations{mod}"]])
    return x
race["test"] = race["test"].map(compute_regard)

In [ ]:
race["test"].to_csv("race_test_continuations_regard_score.tsv", sep="\t")

In [ ]:
race_reg_list = []
for race_cat in set(race["test"]["category"]):
    race_regard=regard.compute(
        data=race["test"].filter(lambda x: x["category"] == race_cat)["continuations_002"],
        aggregation="average")
    race_regard["average_regard"]["race"] = race_cat
    race_reg_list.append(race_regard["average_regard"])
    print(f"Regard for {race_cat} is :{race_regard} ")

In [ ]:
import pandas as pd
race_df = pd.DataFrame(race_reg_list)

In [ ]:
ax = race_df.plot.bar(x='race', stacked=True, figsize=(10,6))
ax.set_title('Regard for different races in text-davinci-002', fontsize=20)
ax.set_ylim(0,1.5)
ax.set_xticklabels(race_df['race'], rotation=0, fontsize=8)
ax.get_figure().savefig ('race-regard-text-davinci-002')

In [ ]:
race_df.head()

In [ ]:
def get_regard(data, race_1, race_2):
    race_1_cont = race["test"].filter(lambda x: x["category"] == race_1)["continuations"]
    race_2_cont = race["test"].filter(lambda x: x["category"] == race_2)["continuations"]
    min_cont = min(len(race_1_cont), len(race_2_cont))
    print(regard.compute(data=race_1_cont[:min_cont], references= race_2_cont[:min_cont], aggregation="average"))

In [ ]:
get_regard(race["test"], 'African_Americans', "European_Americans")

In [ ]:
regard.compute(data = male_continuations, references= female_continuations, aggregation = 'average')